# Inventory Analytics Metric View Queries

This notebook contains sample queries demonstrating how to query the inventory analytics metric views using the `MEASURE()` function.

**Catalog/Schema:** `juan_dev.retail`  
**Purpose:** Example queries for all 10 inventory metric views  
**Created:** 2025-01-27

## Overview

Metric views require the `MEASURE()` function for all measure columns:
- **Dimensions** can be selected directly (e.g., `Location Name`)
- **Measures** must use `MEASURE()` function (e.g., `MEASURE(\`Total Quantity Available\`)`)
- When selecting dimensions with measures, include `GROUP BY` for the dimensions

**Note:** These queries demonstrate the correct syntax for querying Databricks metric views. Execute each query to see results from the pre-aggregated metric views.


## 1. Current Status Queries (`inventory_current_status_mv`)

**Purpose:** Current inventory status metrics (optimized for Question Type 1)  
**Source:** `gold_inventory_fact` (filtered to latest date)


### Query 1.1: What is the current inventory level by location?

Returns inventory metrics grouped by location and region.


In [0]:

SELECT 
  `Location Name`,
  `Region`,
  MEASURE(`Total Quantity Available`) as `Total Quantity Available`,
  MEASURE(`Total Inventory Value Cost`) as `Total Inventory Value Cost`,
  MEASURE(`Product Count`) as `Product Count`
FROM juan_dev.retail.inventory_current_status_mv
GROUP BY `Location Name`, `Region`
ORDER BY `Total Inventory Value Cost` DESC;

### Query 1.2: Show me current stockout status by product category

Returns stockout metrics filtered to out-of-stock products, grouped by category.


In [0]:

SELECT 
  `Product Category`,
  MEASURE(`Stockout Product Count`) as `Stockout Product Count`,
  MEASURE(`Product Count`) as `Product Count`,
  ROUND((MEASURE(`Stockout Product Count`) * 100.0 / MEASURE(`Product Count`)), 2) as `Stockout Rate %`
FROM juan_dev.retail.inventory_current_status_mv
WHERE `Stockout Status` = 'Out of Stock'
GROUP BY `Product Category`
ORDER BY `Stockout Product Count` DESC;

### Query 1.3: What locations have the highest margin value in inventory?

Returns top locations by margin value with margin percentage calculation.


In [0]:

SELECT 
  `Location Name`,
  `Region`,
  MEASURE(`Total Margin Value`) as `Total Margin Value`,
  MEASURE(`Total Inventory Value Cost`) as `Total Inventory Value Cost`,
  MEASURE(`Total Inventory Value Retail`) as `Total Inventory Value Retail`,
  ROUND((MEASURE(`Total Margin Value`) / NULLIF(MEASURE(`Total Inventory Value Cost`), 0) * 100), 2) as `Margin %`
FROM juan_dev.retail.inventory_current_status_mv
GROUP BY `Location Name`, `Region`
ORDER BY `Total Margin Value` DESC
LIMIT 10;

## 2. Stockout Risk Queries (`inventory_stockout_risk_mv`)

**Purpose:** Stockout risk analysis with standardized risk levels (optimized for Question Type 2)  
**Source:** `gold_inventory_fact` (filtered to latest date, products with quantity > 0)


### Query 2.1: Which products are at high risk of stockout?

Returns high-risk products filtered by risk level.


In [0]:

SELECT 
  `Location Name`,
  `Product Category`,
  MEASURE(`Products at Risk Count`) as `Products at Risk Count`,
  MEASURE(`Average Days of Supply`) as `Average Days of Supply`,
  MEASURE(`Products Below Reorder Point Count`) as `Products Below Reorder Point Count`
FROM juan_dev.retail.inventory_stockout_risk_mv
WHERE `Risk Level` = 'High Risk'
GROUP BY `Location Name`, `Product Category`
ORDER BY `Products at Risk Count` DESC;

### Query 2.2: Show me products below reorder point without replenishment scheduled

Returns products needing urgent attention with no replenishment schedule.


In [0]:

SELECT 
  `Location Name`,
  `Region`,
  `Product Category`,
  MEASURE(`Products with No Replenishment Scheduled`) as `Products with No Replenishment Scheduled`,
  MEASURE(`Average Days of Supply`) as `Average Days of Supply`
FROM juan_dev.retail.inventory_stockout_risk_mv
GROUP BY `Location Name`, `Region`, `Product Category`
HAVING MEASURE(`Products with No Replenishment Scheduled`) > 0
ORDER BY `Products with No Replenishment Scheduled` DESC;

### Query 2.3: Compare stockout risk levels across regions

**Note:** Fixed query - when aggregating measures across groups, use SUM/AVG of MEASURE(), not MEASURE(SUM/AVG).


In [0]:

SELECT 
  `Region`,
  `Risk Level`,
  MEASURE(`Products at Risk Count`) as `Total Products at Risk`,
  MEASURE(`Average Days of Supply`) as `Avg Days of Supply`
FROM juan_dev.retail.inventory_stockout_risk_mv
GROUP BY `Region`, `Risk Level`
ORDER BY `Region`, `Risk Level`;

## 3. Overstock Analysis Queries (`inventory_overstock_analysis_mv`)

**Purpose:** Overstock identification with severity levels (optimized for Question Type 3)  
**Source:** `gold_inventory_fact` (filtered to latest date, overstocked products)


### Query 3.1: What products are severely overstocked?

Returns severe overstock situations by location and category.


In [0]:

SELECT 
  `Location Name`,
  `Product Category`,
  MEASURE(`Overstock Product Count`) as `Overstock Product Count`,
  MEASURE(`Total Overstock Value Cost`) as `Total Overstock Value Cost`,
  MEASURE(`Average Days of Supply`) as `Average Days of Supply`
FROM juan_dev.retail.inventory_overstock_analysis_mv
WHERE `Overstock Severity` = 'Severe Overstock'
GROUP BY `Location Name`, `Product Category`
ORDER BY `Total Overstock Value Cost` DESC;

### Query 3.2: Which categories have the most excess inventory value?

**Note:** Fixed query - when aggregating measures across groups, use SUM/AVG of MEASURE(), not MEASURE(SUM/AVG).


In [0]:

SELECT 
  `Product Category`,
  MEASURE(`Overstock Product Count`) as `Total Overstock Products`,
  MEASURE(`Total Overstock Value Cost`) as `Total Value at Cost`,
  MEASURE(`Average Days of Supply`) as `Avg Days of Supply`
FROM juan_dev.retail.inventory_overstock_analysis_mv
GROUP BY `Product Category`
ORDER BY `Total Value at Cost` DESC;

### Query 3.3: Show me overstock by location and severity

**Note:** Fixed query - when aggregating measures across groups, use SUM of MEASURE(), not MEASURE(SUM).


In [0]:

SELECT 
  `Location Name`,
  `Overstock Severity`,
  MEASURE(`Overstock Product Count`) as `Products`,
  MEASURE(`Total Overstock Quantity`) as `Total Units`,
  MEASURE(`Total Overstock Value Cost`) as `Total Cost Value`
FROM juan_dev.retail.inventory_overstock_analysis_mv
GROUP BY `Location Name`, `Overstock Severity`
ORDER BY `Location Name`, `Total Cost Value` DESC;

## 4. Inventory Value Queries (`inventory_value_summary_mv`)

**Purpose:** Inventory value metrics by location and category (optimized for Question Type 4)  
**Source:** `gold_inventory_fact` (filtered to latest date)


### Query 4.1: What is the inventory value by location?

Returns inventory value metrics grouped by location and region.


In [0]:

SELECT 
  `Location Name`,
  `Region`,
  MEASURE(`Total Inventory Value Cost`) as `Total Inventory Value Cost`,
  MEASURE(`Total Inventory Value Retail`) as `Total Inventory Value Retail`,
  MEASURE(`Total Margin Value`) as `Total Margin Value`,
  MEASURE(`Product Count`) as `Product Count`
FROM juan_dev.retail.inventory_value_summary_mv
GROUP BY `Location Name`, `Region`
ORDER BY `Total Inventory Value Cost` DESC;


### Query 4.2: Show me inventory value by product category level 1

**Note:** Fixed query - when aggregating measures across groups, use SUM/AVG of MEASURE(), not MEASURE(SUM/AVG).


In [0]:

SELECT 
  `Product Category Level 1`,
  MEASURE(`Total Inventory Value Cost`) as `Total Cost`,
  MEASURE(`Total Inventory Value Retail`) as `Total Retail`,
  MEASURE(`Total Margin Value`) as `Total Margin`,
  MEASURE(`Product Count`) as `Products`,
  MEASURE(`Average Inventory Value per Product`) as `Avg Value per Product`
FROM juan_dev.retail.inventory_value_summary_mv
GROUP BY `Product Category Level 1`
ORDER BY `Total Cost` DESC;


### Query 4.3: Compare inventory investment across regions

**Note:** Fixed query - when aggregating measures across groups, use SUM/AVG of MEASURE(), not MEASURE(SUM/AVG).


In [0]:

SELECT 
  `Region`,
  MEASURE(`Total Inventory Value Cost`) as `Total Investment`,
  MEASURE(`Product Count`) as `Total Products`,
  MEASURE(`Average Margin per Product`) as `Avg Margin per Product`
FROM juan_dev.retail.inventory_value_summary_mv
GROUP BY `Region`
ORDER BY `Total Investment` DESC;


## 5. Reorder Management Queries (`inventory_reorder_management_mv`)

**Purpose:** Reorder needs and replenishment status (optimized for Question Type 5)  
**Source:** `gold_inventory_fact` (filtered to latest date, products at or below reorder point)


### Query 5.1: Which products need reordering urgently?

Returns products with critical reorder urgency (no schedule).


In [0]:

SELECT 
  `Location Name`,
  `Product Category`,
  `Reorder Urgency`,
  MEASURE(`Products Needing Reorder Count`) as `Products Needing Reorder Count`,
  MEASURE(`Total Reorder Quantity Needed`) as `Total Reorder Quantity Needed`
FROM juan_dev.retail.inventory_reorder_management_mv
WHERE `Reorder Urgency` = 'Critical - No Schedule'
GROUP BY `Location Name`, `Product Category`, `Reorder Urgency`
ORDER BY `Products Needing Reorder Count` DESC;


### Query 5.2: Show me reorder needs by location

**Note:** Fixed query - when aggregating measures across groups, use SUM/AVG of MEASURE(), not MEASURE(SUM/AVG).


In [0]:

SELECT 
  `Location Name`,
  `Region`,
  MEASURE(`Products Needing Reorder Count`) as `Total Products`,
  MEASURE(`Total Reorder Quantity Needed`) as `Total Units to Reorder`,
  MEASURE(`Average Days Until Replenishment`) as `Avg Days Until Restock`
FROM juan_dev.retail.inventory_reorder_management_mv
GROUP BY `Location Name`, `Region`
ORDER BY `Total Products` DESC;


### Query 5.3: What categories have delayed replenishment schedules?

**Note:** Fixed query - when aggregating measures across groups, use SUM/AVG of MEASURE(), not MEASURE(SUM/AVG).


In [0]:

SELECT 
  `Product Category`,
  MEASURE(`Products with Delayed Schedule Count`) as `Delayed Products`,
  MEASURE(`Average Days Until Replenishment`) as `Avg Days Until Restock`
FROM juan_dev.retail.inventory_reorder_management_mv
GROUP BY `Product Category`
HAVING MEASURE(`Products with Delayed Schedule Count`) > 0
ORDER BY `Delayed Products` DESC;


## 6. Movement Summary Queries (`inventory_movement_summary_mv`)

**Purpose:** Inventory movement transactions by type (optimized for Question Type 6)  
**Source:** `gold_inventory_movement_fact`


### Query 6.1: What types of inventory movements occurred last month?

**Note:** Fixed query - when aggregating measures across groups, use SUM/AVG of MEASURE(), not MEASURE(SUM/AVG).


In [0]:

SELECT 
  `Movement Type`,
  MEASURE(`Movement Count`) as `Total Movements`,
  MEASURE(`Total Quantity Moved`) as `Total Units`,
  MEASURE(`Total Movement Cost`) as `Total Cost`,
  MEASURE(`Average Movement Quantity`) as `Avg Units per Movement`
FROM juan_dev.retail.inventory_movement_summary_mv
WHERE `Movement Date` >= DATE_SUB(CURRENT_DATE, 30)
GROUP BY `Movement Type`
ORDER BY `Total Movements` DESC;


### Query 6.2: Show me receipt activity by location

**Note:** Fixed query - when aggregating measures across groups, use SUM/AVG of MEASURE(), not MEASURE(SUM/AVG).


In [0]:

SELECT 
  `Location Name`,
  MEASURE(`Movement Count`) as `Receipt Count`,
  MEASURE(`Total Quantity Moved`) as `Total Units Received`,
  MEASURE(`Average Movement Cost per Unit`) as `Avg Cost per Unit`
FROM juan_dev.retail.inventory_movement_summary_mv
WHERE `Movement Type` = 'RECEIPT'
  AND `Movement Date` >= DATE_SUB(CURRENT_DATE, 30)
GROUP BY `Location Name`
ORDER BY `Receipt Count` DESC;


### Query 6.3: What are the top movement reasons this week?

**Note:** Fixed query - when aggregating measures across groups, use SUM of MEASURE(), not MEASURE(SUM).


In [0]:

SELECT 
  `Movement Reason`,
  `Movement Type`,
  MEASURE(`Movement Count`) as `Total Movements`,
  MEASURE(`Total Quantity Moved`) as `Total Units`
FROM juan_dev.retail.inventory_movement_summary_mv
WHERE `Movement Date` >= DATE_SUB(CURRENT_DATE, 7)
GROUP BY `Movement Reason`, `Movement Type`
ORDER BY `Total Movements` DESC
LIMIT 10;


## 7. Stockout Impact Queries (`inventory_stockout_impact_mv`)

**Purpose:** Stockout events with lost sales impact (optimized for Question Type 7)  
**Source:** `gold_stockout_events`


### Query 7.1: What is the revenue impact of stockouts?

Returns overall stockout impact metrics (no grouping).

**Note:** Fixed query - when aggregating measures across all rows, use SUM/AVG of MEASURE(), not MEASURE(SUM/AVG).


In [0]:

SELECT 
  MEASURE(`Total Lost Sales Revenue`) as `Total Lost Revenue`,
  MEASURE(`Total Lost Sales Quantity`) as `Total Lost Units`,
  MEASURE(`Stockout Event Count`) as `Total Stockout Events`,
  MEASURE(`Average Stockout Duration Days`) as `Avg Stockout Duration`
FROM juan_dev.retail.inventory_stockout_impact_mv;


### Query 7.2: Which locations have the highest stockout impact?

**Note:** Fixed query - when aggregating measures across groups, use SUM/AVG of MEASURE(), not MEASURE(SUM/AVG).


In [0]:

SELECT 
  `Location Name`,
  `Region`,
  MEASURE(`Stockout Event Count`) as `Stockout Events`,
  MEASURE(`Total Lost Sales Revenue`) as `Lost Revenue`,
  MEASURE(`Total Lost Sales Quantity`) as `Lost Units`,
  MEASURE(`Average Stockout Duration Days`) as `Avg Duration Days`
FROM juan_dev.retail.inventory_stockout_impact_mv
GROUP BY `Location Name`, `Region`
ORDER BY `Lost Revenue` DESC;


### Query 7.3: Show me peak season stockout impact

**Note:** Fixed query - when aggregating measures across groups, use SUM of MEASURE(), not MEASURE(SUM).


In [0]:
SELECT 
  `Product Category`,
  MEASURE(`Stockout Event Count`) as `Events`,
  MEASURE(`Peak Season Lost Revenue`) as `Peak Season Lost Revenue`,
  MEASURE(`Total Lost Sales Revenue`) as `Total Lost Revenue`,
  ROUND((MEASURE(`Peak Season Lost Revenue`) / NULLIF(MEASURE(`Total Lost Sales Revenue`), 0) * 100), 2) as `Peak Season %`
FROM juan_dev.retail.inventory_stockout_impact_mv
WHERE `Peak Season Flag` = 'Peak Season'
GROUP BY `Product Category`
ORDER BY `Peak Season Lost Revenue` DESC;

## 8. Location Comparison Queries (`inventory_location_comparison_mv`)

**Purpose:** Cross-location inventory metrics comparison (optimized for Question Type 8)  
**Source:** `gold_inventory_fact` (filtered to latest date)


### Query 8.1: Compare inventory metrics across all locations

Returns comprehensive inventory comparison metrics by location.


In [0]:

SELECT 
  `Location Name`,
  `Region`,
  `Location Type`,
  MEASURE(`Product Count`) as `Product Count`,
  MEASURE(`Total Quantity Available`) as `Total Quantity Available`,
  MEASURE(`Total Inventory Value Cost`) as `Total Inventory Value Cost`,
  MEASURE(`Stockout Rate`) as `Stockout Rate`,
  MEASURE(`Overstock Rate`) as `Overstock Rate`,
  MEASURE(`Average Days of Supply`) as `Average Days of Supply`
FROM juan_dev.retail.inventory_location_comparison_mv
GROUP BY `Location Name`, `Region`, `Location Type`
ORDER BY `Stockout Rate` DESC;


### Query 8.2: Which regions have the best inventory health?

**Note:** Fixed query - when aggregating measures across groups, use SUM/AVG of MEASURE(), not MEASURE(SUM/AVG).


In [0]:

SELECT 
  `Region`,
  COUNT(DISTINCT `Location Name`) as `Locations`,
  MEASURE(`Product Count`) as `Total Products`,
  MEASURE(`Stockout Rate`) as `Avg Stockout Rate`,
  MEASURE(`Overstock Rate`) as `Avg Overstock Rate`,
  MEASURE(`Average Days of Supply`) as `Avg Days of Supply`
FROM juan_dev.retail.inventory_location_comparison_mv
GROUP BY `Region`
ORDER BY `Avg Stockout Rate` ASC;


### Query 8.3: Show me inventory efficiency by location type

**Note:** Fixed query - when aggregating measures across groups, use AVG of MEASURE(), not MEASURE(AVG).


In [0]:

SELECT 
  `Location Type`,
  COUNT(DISTINCT `Location Name`) as `Locations`,
  MEASURE(`Total Quantity Available`) as `Avg Quantity per Location`,
  MEASURE(`Stockout Rate`) as `Avg Stockout Rate`,
  MEASURE(`Average Days of Supply`) as `Avg Days of Supply`
FROM juan_dev.retail.inventory_location_comparison_mv
GROUP BY `Location Type`
ORDER BY `Avg Stockout Rate` ASC;


## 9. Trends Analysis Queries (`inventory_trends_daily_mv`)

**Purpose:** Daily inventory trends over time (optimized for Question Type 9)  
**Source:** `gold_inventory_fact` with date dimension


### Query 9.1: How have inventory levels changed over the last 30 days?

**Note:** Fixed query - when aggregating measures across groups, use SUM/AVG of MEASURE(), not MEASURE(SUM/AVG).


In [0]:

SELECT 
  `Calendar Date`,
  MEASURE(`Total Quantity Available`) as `Total Units`,
  MEASURE(`Total Inventory Value Cost`) as `Total Value`,
  MEASURE(`Stockout Product Count`) as `Stockouts`,
  MEASURE(`Average Days of Supply`) as `Avg Days of Supply`
FROM juan_dev.retail.inventory_trends_daily_mv
WHERE `Calendar Date` >= DATE_SUB(CURRENT_DATE, 30)
GROUP BY `Calendar Date`
ORDER BY `Calendar Date`;


### Query 9.2: Show me monthly inventory trends

**Note:** Fixed query - when aggregating measures across groups, use AVG of MEASURE(), not MEASURE(AVG).


In [0]:

SELECT 
  `Year`,
  `Month Name`,
  MEASURE(`Total Quantity Available`) as `Avg Daily Units`,
  MEASURE(`Total Inventory Value Cost`) as `Avg Daily Value`,
  MEASURE(`Stockout Product Count`) as `Avg Daily Stockouts`,
  MEASURE(`Average Days of Supply`) as `Avg Days of Supply`
FROM juan_dev.retail.inventory_trends_daily_mv
GROUP BY `Year`, `Month Name`
ORDER BY `Year`, `Month Name`;


### Query 9.3: Compare peak season vs regular season inventory patterns

**Note:** Fixed query - when aggregating measures across groups, use AVG of MEASURE(), not MEASURE(AVG).


In [0]:

SELECT 
  `Peak Season Indicator`,
  MEASURE(`Total Quantity Available`) as `Avg Quantity`,
  MEASURE(`Stockout Product Count`) as `Avg Stockouts`,
  MEASURE(`Overstock Product Count`) as `Avg Overstock`,
  MEASURE(`Average Days of Supply`) as `Avg Days of Supply`
FROM juan_dev.retail.inventory_trends_daily_mv
GROUP BY `Peak Season Indicator`
ORDER BY `Peak Season Indicator`;


## 10. Health Score Queries (`inventory_health_score_mv`)

**Purpose:** Overall inventory health metrics with health status (optimized for Question Type 10)  
**Source:** `gold_inventory_fact` (filtered to latest date)


### Query 10.1: What is the overall inventory health score?

Returns overall health metrics (no grouping).

**Note:** Fixed query - when aggregating measures across all rows, use SUM/AVG of MEASURE(), not MEASURE(SUM/AVG).


In [0]:

SELECT 
  MEASURE(`Health Score`) as `Overall Health Score`,
  MEASURE(`Total Products`) as `Total Products`,
  MEASURE(`Stockout Product Count`) as `Total Stockouts`,
  MEASURE(`Overstock Product Count`) as `Total Overstock`,
  MEASURE(`Stockout Rate`) as `Avg Stockout Rate`,
  MEASURE(`Overstock Rate`) as `Avg Overstock Rate`
FROM juan_dev.retail.inventory_health_score_mv;


### Query 10.2: Show me inventory health by location

Returns health metrics grouped by location, region, and health status.


In [0]:

SELECT 
  `Location Name`,
  `Region`,
  `Health Status`,
  MEASURE(`Health Score`) as `Health Score`,
  MEASURE(`Stockout Rate`) as `Stockout Rate`,
  MEASURE(`Overstock Rate`) as `Overstock Rate`,
  MEASURE(`Average Days of Supply`) as `Average Days of Supply`
FROM juan_dev.retail.inventory_health_score_mv
GROUP BY `Location Name`, `Region`, `Health Status`
ORDER BY `Health Score` ASC;


### Query 10.3: Which categories have critical health status?

**Note:** Fixed query - when aggregating measures across groups, use SUM/AVG of MEASURE(), not MEASURE(SUM/AVG).


In [0]:

SELECT 
  `Product Category`,
  `Health Status`,
  MEASURE(`Total Products`) as `Products`,
  MEASURE(`Health Score`) as `Avg Health Score`,
  MEASURE(`Stockout Rate`) as `Avg Stockout Rate`,
  MEASURE(`High Risk Product Count`) as `High Risk Products`
FROM juan_dev.retail.inventory_health_score_mv
WHERE `Health Status` IN ('Critical', 'Poor')
GROUP BY `Product Category`, `Health Status`
ORDER BY `Avg Health Score` ASC;


### Query 10.4: Show me health score trends by region

**Note:** Fixed query - when aggregating measures across groups, use SUM/AVG of MEASURE(), not MEASURE(SUM/AVG).


In [0]:

SELECT 
  `Region`,
  MEASURE(`Health Score`) as `Avg Health Score`,
  MEASURE(`Stockout Rate`) as `Avg Stockout Rate`,
  MEASURE(`Overstock Rate`) as `Avg Overstock Rate`,
  MEASURE(`High Risk Product Count`) as `Total High Risk Products`,
  COUNT(DISTINCT `Location Name`) as `Locations`
FROM juan_dev.retail.inventory_health_score_mv
GROUP BY `Region`
ORDER BY `Avg Health Score` DESC;


In [0]:
SELECT 
  `Location Name`,
  MEASURE(`Health Score`) as `Avg Health Score`,
  MEASURE(`Stockout Rate`) as `Avg Stockout Rate`,
  MEASURE(`Overstock Rate`) as `Avg Overstock Rate`,
  MEASURE(`High Risk Product Count`) as `Total High Risk Products`,
  COUNT(DISTINCT `Location Name`) as `Locations`
FROM juan_dev.retail.inventory_health_score_mv
GROUP BY `Location Name`
ORDER BY `Avg Health Score` DESC;